In [1]:
import os, shutil
import csv, json
import numpy as np
import pandas as pd

In [2]:
def process_csv(filename):
    example_file = open(filename, encoding="utf-8")
    example_reader = csv.reader(example_file)
    example_data = list(example_reader)[3:]
    example_file.close()
    return example_data

def get_column_desc(data):
    column_desc = []
    i = 0
    for idx in range(len(data)):
        raw_data = data[idx]
        # get column names and their description
        if len(raw_data) and "# COLUMN" in raw_data[0]:
            splitted = raw_data[0].split(":")
            column_desc.append([splitted[0].split("# COLUMN ")[1], splitted[1].strip()])
        if '#' in raw_data[0]: # keep track of the row idx of the header row
            i += 1
        else:
            break
        
    return (column_desc, i)

csv_raw = process_csv("nasa_raw.csv") # read the dataset
csv_col_desc, i = get_column_desc(csv_raw) # extract the mapping of col_ids to col_names from dataset
csv_col_desc[:5]

[['pl_name', 'Planet Name'],
 ['hostname', 'Host Name'],
 ['pl_letter', 'Planet Letter'],
 ['hd_name', 'HD ID'],
 ['hip_name', 'HIP ID']]

In [3]:
csv_data = csv_raw[i:] # slice off the mapping of col_ids to col_names to get to the real dataset
csv_header = csv_data[0] # slice off the header
csv_rows = csv_data[1:] # get the real dataset
len(csv_rows)

5602

In [4]:
all_df = pd.DataFrame(csv_rows, columns=csv_header) # convert dataset into DataFrame
all_df = all_df.replace('', None) # replace all missing data with NaN
all_df.rename(columns = dict(csv_col_desc), inplace = True) # replace col_ids in columns with actual col_names
all_df.rename(columns = {"Planet Mass or Mass*sin(i) [Earth Mass]": "Planet Mass [Earth Mass]", # edit typos in col_names
                         "Orbit Semi-Major Axis [au])": "Orbit Semi-Major Axis [au]"}, inplace = True)
# replace inconsistent ' ' in column 'Spectral Type'
all_df['Spectral Type'] = all_df['Spectral Type'].apply(lambda x: str(x).replace(" ", "")).replace('None', None)
all_df.head()

,rowid,Planet Name,Host Name,Planet Letter,HD ID,HIP ID,TIC ID,GAIA ID,Number of Stars,Number of Planets,...,TESS Magnitude,TESS Magnitude Reference,Kepler Magnitude,Kepler Magnitude Reference,Number of Notes,Number of Photometry Time Series,Number of Radial Velocity Time Series,Number of Stellar Spectra Measurements,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,2,1,...,3.8379000,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,None,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,2,1,2,0,0,0
1,2,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,...,3.8229400,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,None,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,0,1,1,0,0,0
2,3,14 And b,14 And,b,HD 221345,HIP 116076,TIC 333225860,Gaia DR2 1920113512486282240,1,1,...,4.3214000,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,None,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,0,1,1,0,0,0
3,4,14 Her b,14 Her,b,HD 145675,HIP 79248,TIC 219483057,Gaia DR2 1385293808145621504,1,2,...,5.8631000,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,None,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,0,1,4,1,0,0
4,5,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,TIC 27533327,Gaia DR2 2135550755683407232,3,1,...,5.6281000,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,6.095,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,5,1,4,3,0,0


In [5]:
# columns we want in the `planets` files
PLANET_COLUMNS = ['Planet Name',
                'Discovery Method', 'Discovery Year', 
                'Controversial Flag', 'Orbital Period [days]', 
                'Planet Radius [Earth Radius]', 
                'Planet Mass [Earth Mass]',
                  'Orbit Semi-Major Axis [au]',
                 'Eccentricity', 
                  'Equilibrium Temperature [K]',
                  'Insolation Flux [Earth Flux]']

In [6]:
planets_df = all_df.loc[:, PLANET_COLUMNS] # extract only the columns needed for planets
planets_df.head()

,Planet Name,Discovery Method,Discovery Year,Controversial Flag,Orbital Period [days],Planet Radius [Earth Radius],Planet Mass [Earth Mass],Orbit Semi-Major Axis [au],Eccentricity,Equilibrium Temperature [K],Insolation Flux [Earth Flux]
0,11 Com b,Radial Velocity,2007,0,323.21000000,12.200,4914.89849,1.178000,0.238000,None,None
1,11 UMi b,Radial Velocity,2009,0,516.21997000,12.300,4684.81420,1.530000,0.080000,None,None
2,14 And b,Radial Velocity,2008,0,186.76000000,13.100,1131.15130,0.775000,0.000000,None,None
3,14 Her b,Radial Velocity,2002,0,1765.03890000,12.600,2559.47216,2.774000,0.373000,None,None
4,16 Cyg B b,Radial Velocity,1996,0,798.50000000,13.500,565.73740,1.660000,0.680000,None,None


In [7]:
# columns we want in the `stars` files
STAR_COLUMNS = ['Host Name', 'Spectral Type', 
                'Stellar Effective Temperature [K]', 
                'Stellar Radius [Solar Radius]', 
                'Stellar Mass [Solar mass]', 
                'Stellar Luminosity [log(Solar)]', 
                'Stellar Surface Gravity [log10(cm/s**2)]', 
                'Stellar Age [Gyr]']

In [8]:
stars_df = all_df.loc[:, STAR_COLUMNS] # extract only the columns needed for stars
stars_df.rename(columns = {"Host Name": "Star Name"}, inplace = True) # change column 'Host Name' to 'Star Name'

# since some stars appear several times, we group the data by the unique star names, drop None values and
# pick a random value amongst the remaining values for each Star
stars_df = stars_df.groupby(['Star Name'])[STAR_COLUMNS[1:]].agg(lambda x: np.random.choice(x.mode(dropna=False)))
stars_df = stars_df.reset_index()
stars_df.head()

,Star Name,Spectral Type,Stellar Effective Temperature [K],Stellar Radius [Solar Radius],Stellar Mass [Solar mass],Stellar Luminosity [log(Solar)],Stellar Surface Gravity [log10(cm/s**2)],Stellar Age [Gyr]
0,11 Com,G8III,4874.00,13.76,2.09,1.978,2.45,None
1,11 UMi,K4III,4213.00,29.79,2.78,2.430,1.93,1.560
2,14 And,K0III,4888.00,11.55,1.78,1.840,2.55,4.500
3,14 Her,K0V,5338.00,0.93,0.91,-0.153,4.45,3.900
4,16 Cyg B,G3V,5750.00,1.13,1.08,0.097,4.36,7.400


In [9]:
planets_n_stars_df = all_df.loc[:, ["Planet Name", "Host Name"]] # extract just the Planet Name - Host Name mapping
planets_n_stars_df

,Planet Name,Host Name
0,11 Com b,11 Com
1,11 UMi b,11 UMi
2,14 And b,14 And
3,14 Her b,14 Her
4,16 Cyg B b,16 Cyg B
...,...,...
5597,ups And b,ups And
5598,ups And c,ups And
5599,ups And d,ups And
5600,ups Leo b,ups Leo


In [10]:
# find how many stars have how many planets

stars_planets_count = planets_n_stars_df.groupby("Host Name", as_index = False).count()
stars_planets_count = stars_planets_count.rename(columns = {"Planet Name": "num_planets_per_star"})
stars_planets_count = stars_planets_count.groupby("num_planets_per_star", as_index = False).count()
stars_planets_count = stars_planets_count.rename(columns = {"Host Name": "num_stars"})
stars_planets_count["total_num_planets"] = stars_planets_count["num_planets_per_star"] * stars_planets_count["num_stars"]
stars_planets_count

,num_planets_per_star,num_stars,total_num_planets
0,1,3221,3221
1,2,626,1252
2,3,203,609
3,4,76,304
4,5,27,135
5,6,11,66
6,7,1,7
7,8,1,8


#### Break data into files:

1. stars that only have one planet (1) (size ~ 1500) : `stars_1.csv`
2. planets of these hosts included in : `planets_1.csv`
3. mappings from planets to hosts for these planets and hosts in : `mappping_1.json`
4. stars that only have one planet (2) (size ~ 1500) : `stars_2.csv`
5. planets of these hosts included in : `planets_2.csv`
6. mappings from planets to hosts for these planets and hosts in : `mappping_1.json`
7. stars that only have two planets (size ~ 600) : `stars_3.csv`
8. planets of these hosts included in : `planets_3.csv`
9. mappings from planets to hosts for these planets and hosts in : `mappping_1.json`
10. stars that have 3 planets (size ~ 300) : `stars_4.csv`
11. planets of these hosts included in : `planets_4.csv`
12. mappings from planets to hosts for these planets and hosts in : `mappping_1.json`
13. stars that have >= 4 planets (size ~ 125) : `stars_5.csv`
14. planets of these hosts included in : `planets_5.csv`
15. mappings from planets to hosts for these planets and hosts in : `mappping_1.json`

In [11]:
# find out how many planets each Host star has

host_planet_count = planets_n_stars_df.groupby("Host Name", as_index = True).count()[["Planet Name"]]
host_planet_count = host_planet_count.rename(columns = {"Planet Name": "Number of Planets"})
host_planet_count = host_planet_count.reset_index()
host_planet_count.head()

,Host Name,Number of Planets
0,11 Com,1
1,11 UMi,1
2,14 And,1
3,14 Her,2
4,16 Cyg B,1


In [12]:
# make sets of stars that appear in each `stars` file

stars = {}
stars_with_1_planet = list(host_planet_count[host_planet_count['Number of Planets'] == 1]['Host Name'])
num_stars_1_planet = len(stars_with_1_planet)
stars[1] = set(stars_with_1_planet[:num_stars_1_planet//2])
stars[2] = set(stars_with_1_planet[num_stars_1_planet//2:])
stars[3] = set(host_planet_count[host_planet_count['Number of Planets'] == 2]['Host Name'])
stars[4] = set(host_planet_count[host_planet_count['Number of Planets'] == 3]['Host Name'])
stars[5] = set(host_planet_count[host_planet_count['Number of Planets'] >= 4]['Host Name'])
{i: len(stars[i]) for i in stars}

{1: 1610, 2: 1611, 3: 626, 4: 203, 5: 116}

In [13]:
def get_data_for_stars(stars, stars_df=stars_df, planets_df=planets_df, planets_n_stars_df=planets_n_stars_df):
    '''input: set of stars (Star Names)
       output: dict storing DataFrames of Star data, Planet data, and Planet - Star mapping for given `stars`'''
    stars_df = stars_df[stars_df['Star Name'].isin(stars)]
    planets_n_stars_df = planets_n_stars_df[planets_n_stars_df['Host Name'].isin(stars)]
    planets = set(planets_n_stars_df['Planet Name'])
    planets_df = planets_df[planets_df['Planet Name'].isin(planets)]
    planets_n_stars_df.set_index("Planet Name", inplace = True)
    planets_n_stars_df = planets_n_stars_df['Host Name']
    return {'stars': stars_df, 'planets': planets_df, 'mapping': planets_n_stars_df}

In [14]:
# write correct data into files in `unbroken_data` directory

if os.path.exists('unbroken_data'):
    shutil.rmtree('unbroken_data')
os.mkdir('unbroken_data')

for i in stars:
    data = get_data_for_stars(stars[i])
    data['stars'].to_csv(os.path.join("unbroken_data", 'stars_%d.csv' % (i)), index=False, encoding='utf-8')
    data['planets'].to_csv(os.path.join("unbroken_data", 'planets_%d.csv' % (i)), index=False, encoding='utf-8')
    with open(os.path.join("unbroken_data", 'mapping_%d.json' % (i)), 'w', encoding='utf-8') as mapping_file:
        data['mapping'].to_json(mapping_file, force_ascii=False)